In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Flatten
from keras. layers import MaxPooling2D
from keras.layers import Dense, Dropout
import numpy as np

In [ ]:
model=Sequential()
conv1=model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
conv1=model.add(Conv2D(64,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.4,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set =train_datagen.flow_from_directory('Training',target_size = (64, 64),batch_size = 6,class_mode = 'binary')
test_set = test_datagen.flow_from_directory('Testing',target_size = (64, 64),batch_size = 6,class_mode = 'binary')

In [ ]:
model.fit(training_set,steps_per_epoch = 20,epochs = 50)

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model

model = load_model('model.h5')

img_path = input("Enter image path: ")

test_image = image.load_img(img_path, target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = model.predict(test_image)
prediction = 'No Glasses' if result[0][0] == 1 else 'Glasses'
print("Prediction:", prediction)


In [ ]:
model.save("model.h5")

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

# Load the pre-trained Keras model
model = load_model('model.h5')

# Load the model class indices
class_indices = training_set.class_indices

# Function to make a prediction based on the model
def make_prediction(image_array):
    result = model.predict(image_array)
    if result[0][0] == 1:
        prediction = 'Glasses'
    else:
        prediction = 'No Glasses'
    return prediction

# Function to capture a live image and make a prediction
def capture_and_predict():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        cv2.imshow('Live Image', frame)

        # Process the frame for the model (resize and normalize)
        processed_frame = cv2.resize(frame, (64, 64))
        processed_frame = processed_frame.astype('float32') / 255.0
        processed_frame = np.expand_dims(processed_frame, axis=0)

        # Make a prediction using the model
        prediction = make_prediction(processed_frame)

        # Display the prediction on the frame
        cv2.putText(frame, prediction, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Live Image with Prediction', frame)

        # Exit the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and close the OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function to capture live image and make predictions
capture_and_predict()
